In [2]:
import tensorflow as tf
import numpy as np
import os

tf.logging.set_verbosity(tf.logging.INFO)

FLAGS = tf.app.flags.FLAGS

tf.app.flags.DEFINE_integer('batch_size', 128,"Number of images to process in a batch.")
tf.app.flags.DEFINE_integer('epoch', 40,"Number of epoch")
tf.app.flags.DEFINE_integer('word_embedding_size', 300)
tf.app.flags.DEFINE_integer('image_class_num', 100, "Number of image classes")
tf.app.flags.DEFINE_float('margin', 0.1)

# 100 labels of cifar-100
CLASSES = []

def weight_variable(shape, w=0.1):
    initial = tf.truncated_normal(shape, stddev=w) #Outputs random values from a truncated normal distribution.
    return tf.Variable(initial)

def bias_variable(shape, w=0.1):
    initial = tf.constant(w, shape=shape)
    return tf.Variable(initial)

# Giving a tensor return a tensor of embedding from lookup table
# Input: labels : a set of strings
def embedding_lookup(labels):
    
    return embedding

def nearest_neighbor_labels(inputs):
    labels = empty((inputs.shape[0], 0))
    return labels

def devise(features, labels, mode):
    ########## Core Visual Model ##########
    print('input data shape: ', features["x"])
    input_layer = tf.reshape(features["x"], [-1, 32, 32, 3])
    print('input layer shape: ',input_layer.shape)
    # conv1
    with tf.variable_scope('conv1') as scope:
        kernel = weight_variable(shape=[5, 5, 3, 64]) #shape=[filter_height * filter_width * in_channels, output_channels]
        conv = tf.nn.conv2d(input_layer, kernel, [1, 1, 1, 1], padding='SAME')
        biases = bias_variable(shape=[64], w=0.0)
        pre_activation = tf.nn.bias_add(conv, biases)
        conv1 = tf.nn.relu(pre_activation, name=scope.name)
       
    # pool1
    pool1 = tf.nn.max_pool(conv1, ksize=[1, 3, 3, 1], strides=[1, 2, 2, 1],
                         padding='SAME', name='pool1')
    print('pool1 shape: ', pool1)
    # norm1
    norm1 = tf.nn.lrn(pool1, 4, bias=1.0, alpha=0.001 / 9.0, beta=0.75, name='norm1')

    # conv2
    with tf.variable_scope('conv2') as scope:
        kernel = weight_variable(shape=[5, 5, 64, 64])
        conv = tf.nn.conv2d(norm1, kernel, [1, 1, 1, 1], padding='SAME')
        biases = bias_variable(shape=[64], w=0.1)
        pre_activation = tf.nn.bias_add(conv, biases)
        conv2 = tf.nn.relu(pre_activation, name=scope.name)

    # norm2
    norm2 = tf.nn.lrn(conv2, 4, bias=1.0, alpha=0.001 / 9.0, beta=0.75,
                    name='norm2')
    # pool2
    pool2 = tf.nn.max_pool(norm2, ksize=[1, 3, 3, 1],
                         strides=[1, 2, 2, 1], padding='SAME', name='pool2')
    print('pool2 shape: ', pool2)
    
    pool2_flat = tf.reshape(pool2, [-1, 8*8*64])
        dense1 = tf.layers.dense(
        inputs=pool2_flat,
        units=1024, # number of neurons in the dense layer
        activation=tf.nn.relu,
        name='dense1')
    dropout1 = tf.layers.dropout(
        inputs=dense1,
        rate=0.4,
        training= mode==tf.estimator.ModeKeys.TRAIN,
        name='dropout1')
    dense2 = tf.layers.dense(
        inputs=dropout1,
        units=1024, # number of neurons in the dense layer
        activation=tf.nn.relu,
        name='dense2')
    dropout2 = tf.layers.dropout(
        inputs=dense2,
        rate=0.4,
        training= mode==tf.estimator.ModeKeys.TRAIN,
        name='dropout2')
    ########## Core Visual Model ##########
    
    ########## Transformation ##########
    transform = tf.layers.dense(inputs=dropout2, unit=FLAGS.word_embedding_size)
    ########## Transformation ##########
    
    predictions = {
        "embedding": trainsform,
        "labels": nearest_neighbor_labels(transform)
    }
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.ExtimatorSpec(mode=modem, predictions=predictions)

    # Calculate loss
    loss = 0
    margin = FLAGS.margin
    for label in labels:
        for i in range(FLAGS.image_class_num):
            if label != i:
                lo = margin - tf.matmul(embedding_lookup(CLASSES[label]),tf.matmul(transform,dropout2)) + tf.matmul(embedding_lookup(CLASSES[i]),tf.matmul(transform,dropout2))
                truncat = max(0, lo)
                if truncat == 0: #??
                    break
                loss+=truncat
    # TRAIN mode
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.GradientDescentOptimizer(0.001)
        train_op = optimizer.minimize(
            loss = loss,
            global_step = tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)
    # EVEL mode
    eval_metric_ops = {
        "accuracy": tf.metrics.accuracy( #??
          labels=labels, predictions=predictions["labels"])}
    return tf.estimator.EstimatorSpec(
      mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)

def main(unused_argv):
    ########## Data ##########
    # Data format:
    # data -- a 10000x3072 numpy array of uint8s. 
    #         Each row of the array stores a 32x32 colour image. 
    #         The first 1024 entries contain the red channel values, 
    #         the next 1024 the green, and the final 1024 the blue. 
    #         The image is stored in row-major order, so that the first 32 entries of the array are the red channel values of the first row of the image.
    # labels -- a list of 10000 numbers in the range 0-99. The number at index i indicates the label of the ith image in the array data.

    # Load training data
    train_set = unpickle('./Data/cifar-100/train')
    train_data = np.asarray(train_set[b'data'], dtype=np.float32)

    ########## Data ##########
    
    ########## CNN classifier ##########
    cifar_classifier = tf.estimator.Estimator(model_fn=cnn_model_fn, model_dir="./tmp/cifar100",
                                              config=tf.estimator.RunConfig().replace(save_summary_steps=10))
    ########## CNN classifier ##########
    
    ########## Train ##########
    # Set up logging for predictions
    tensors_to_log = {"probabilities": "softmax_tensor"}
    logging_hook = tf.train.LoggingTensorHook(
          tensors=tensors_to_log, every_n_iter=50)
    
    print('batch size: ',FLAGS.batch_size)
    # Train the model
    train_input_fn = tf.estimator.inputs.numpy_input_fn(
        x={"x": train_data},
        y=train_labels,
        batch_size=FLAGS.batch_size, # number of data in a minibatch
        num_epochs=FLAGS.epoch,
        shuffle=True) # shuffle training data
    cifar_classifier.train(
        input_fn=train_input_fn,
        steps=100, #number of times the training loop in your learning algorithm will run to update the parameters in the model. In each loop iteration, it will process a chunk of data, which is basically a batch. Usually, this loop is based on the Gradient Descent algorithm.
        hooks=[logging_hook])
    ########## Train ##########
    
    ########## Evaluate ##########
    # Evaluate the model and print results
    eval_input_fn = tf.estimator.inputs.numpy_input_fn(
        x={"x": eval_data},
        y=eval_labels,
        num_epochs=1,
        shuffle=False)
    eval_results = cifar_classifier.evaluate(input_fn=eval_input_fn)
    print(eval_results)
    ########## Evaluate ##########
if __name__ == "__main__":
    tf.app.run()


SyntaxError: unexpected EOF while parsing (<ipython-input-2-40bd720c000f>, line 19)